<a href="https://colab.research.google.com/github/Akash9356/Akash9356/blob/main/Automatic_Question_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation of libraries

In [ ]:
!pip install --quiet transformers==4.34.0

time: 7.35 s (started: 2023-12-14 11:23:14 +00:00)


In [ ]:
!pip install --quiet sentencepiece==0.1.99

time: 6.33 s (started: 2023-12-14 11:23:21 +00:00)


In [ ]:
!pip install --quiet textwrap3==0.9.2

time: 7.74 s (started: 2023-12-14 11:23:28 +00:00)


In [ ]:
!pip install --quiet nltk==3.8.0

time: 6.83 s (started: 2023-12-14 11:23:36 +00:00)


In [ ]:
!pip install --quiet ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.17 s (started: 2023-12-14 11:23:42 +00:00)


## **Input**


In [ ]:
from textwrap3 import wrap

text = """In the 16th century, an age of great marine and terrestrial exploration, Ferdinand Magellan led the first expedition to sail around the world. As a young Portuguese noble, he served the king of Portugal, but he became involved in the quagmire of political intrigue at court and lost the king’s favor. After he was dismissed from service by the king of Portugal, he offered to serve the future Emperor Charles V of Spain.

A papal decree of 1493 had assigned all land in the New World west of 50 degrees W longitude to Spain and all the land east of that line to Portugal. Magellan offered to prove that the East Indies fell under Spanish authority. On September 20, 1519, Magellan set sail from Spain with five ships. More than a year later, one of these ships was exploring the topography of South America in search of a water route across the continent. This ship sank, but the remaining four ships searched along the southern peninsula of South America. Finally they found the passage they sought near 50 degrees S latitude. Magellan named this passage the Strait of All Saints, but today it is known as the Strait of Magellan.

One ship deserted while in this passage and returned to Spain, so fewer sailors were privileged to gaze at that first panorama of the Pacific Ocean. Those who remained crossed the meridian now known as the International Date Line in the early spring of 1521 after 98 days on the Pacific Ocean. During those long days at sea, many of Magellan’s men died of starvation and disease.

Later, Magellan became involved in an insular conflict in the Philippines and was killed in a tribal battle. Only one ship and 17 sailors under the command of the Basque navigator Elcano survived to complete the westward journey to Spain and thus prove once and for all that the world is round, with no precipice at the edge."""

for wrp in wrap(text, 150):
  print (wrp)
print ("\n")

In the 16th century, an age of great marine and terrestrial exploration, Ferdinand Magellan led the first expedition to sail around the world. As a
young Portuguese noble, he served the king of Portugal, but he became involved in the quagmire of political intrigue at court and lost the king’s
favor. After he was dismissed from service by the king of Portugal, he offered to serve the future Emperor Charles V of Spain.  A papal decree of 1493
had assigned all land in the New World west of 50 degrees W longitude to Spain and all the land east of that line to Portugal. Magellan offered to
prove that the East Indies fell under Spanish authority. On September 20, 1519, Magellan set sail from Spain with five ships. More than a year later,
one of these ships was exploring the topography of South America in search of a water route across the continent. This ship sank, but the remaining
four ships searched along the southern peninsula of South America. Finally they found the passage they sought 

# **Summarization with T5**

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_model = summary_model.to(device)


time: 5.1 s (started: 2023-12-14 11:23:52 +00:00)


In [ ]:
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

time: 5.32 ms (started: 2023-12-14 11:23:57 +00:00)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize

def postprocesstext (content):
  final=""
  for sent in sent_tokenize(content):
    sent = sent.capitalize()
    final = final +" "+sent
  return final


def summarizer(text,model,tokenizer):
  text = text.strip().replace("\n"," ")
  text = "summarize: "+text
  # print (text)
  max_len = 512
  encoding = tokenizer.encode_plus(text,max_length=max_len,
                                   pad_to_max_length=False,
                                   truncation=True,
                                   return_tensors="pt").to(device)

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=3,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  min_length = 75,
                                  max_length=300)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
  summary = dec[0]
  summary = postprocesstext(summary)
  summary= summary.strip()

  return summary


summarized_text = summarizer(text,summary_model,summary_tokenizer)


print ("\noriginal Text >>")
for wrp in wrap(text, 150):
  print (wrp)
print ("\n")
print ("Summarized Text >>")
for wrp in wrap(summarized_text, 150):
  print (wrp)
print ("\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



original Text >>
In the 16th century, an age of great marine and terrestrial exploration, Ferdinand Magellan led the first expedition to sail around the world. As a
young Portuguese noble, he served the king of Portugal, but he became involved in the quagmire of political intrigue at court and lost the king’s
favor. After he was dismissed from service by the king of Portugal, he offered to serve the future Emperor Charles V of Spain.  A papal decree of 1493
had assigned all land in the New World west of 50 degrees W longitude to Spain and all the land east of that line to Portugal. Magellan offered to
prove that the East Indies fell under Spanish authority. On September 20, 1519, Magellan set sail from Spain with five ships. More than a year later,
one of these ships was exploring the topography of South America in search of a water route across the continent. This ship sank, but the remaining
four ships searched along the southern peninsula of South America. Finally they found the pa

# **Answer Span Extraction (Keywords and Noun Phrases)**

In [ ]:
!pip install --quiet git+https://github.com/boudinfl/pke.git
!pip install --quiet flashtext==2.7

  Preparing metadata (setup.py) ... done
time: 24.5 s (started: 2023-12-14 11:24:23 +00:00)


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback

def get_nouns_multipartite(content):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=content)
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'PROPN','NOUN'}
        #pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos, stoplist=stoplist)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=15)


        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

time: 3.04 ms (started: 2023-12-14 11:24:47 +00:00)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pip install --upgrade spacy

time: 8.05 s (started: 2023-12-14 11:24:47 +00:00)


In [ ]:
!python -m spacy download en_core_web_sm

2023-12-14 11:25:00.390229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 11:25:00.390305: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 11:25:00.390356: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 11:25:01.808912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
time: 28.9 s (started: 2023-12-14 11:24:55 +00:00)


In [ ]:
import spacy

def extract_keywords(original_text, summarized_text):
    nlp = spacy.load("en_core_web_sm")

    # Process the original and summarized texts
    doc_original = nlp(original_text)
    doc_summarized = nlp(summarized_text)

    # Get important tokens from the original text
    important_original_tokens = {token.text.lower() for token in doc_original
                                 if not token.is_stop and token.is_alpha}

    # Get important tokens from the summarized text
    important_summarized_tokens = {token.text.lower() for token
                                   in doc_summarized if not
                                   token.is_stop and token.is_alpha}

    # Find the common tokens between the two sets
    common_keywords = list(important_original_tokens.intersection(important_summarized_tokens))

    return common_keywords[:10]


common_keywords = extract_keywords(text, summarized_text)
print(common_keywords)


['topography', 'route', 'new', 'assigned', 'decree', 'continent', 'magellan', 'exploring', 'south', 'land']
time: 2.51 s (started: 2023-12-14 11:25:24 +00:00)


# **Question generation with T5**

In [ ]:
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

time: 4.17 s (started: 2023-12-14 11:25:27 +00:00)


In [ ]:
def get_question(context,answer,model,tokenizer):
  text = "context: {} answer: {}".format(context,answer)
  encoding = tokenizer.encode_plus(text,max_length=384,
                                   pad_to_max_length=False,truncation=True,
                                   return_tensors="pt").to(device)
  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question



for wrp in wrap(summarized_text, 150):
  print (wrp)
print ("\n")

for answer in common_keywords:
  ques = get_question(summarized_text,answer,question_model,question_tokenizer)
  print (ques)
  print (answer.capitalize())
  print ("\n")


A papal decree of 1493 had assigned all land in the new world west of 50 degrees w longitude to spain and all the land east of that line to portugal.
On september 20, 1519, magellan set sail from spain with five ships and one of them was exploring the topography of south america in search of water
route across continent. This ship sank, but the remaining four ships searched along the southern peninsula of south america.


What area of south america was one of the ships exploring?
Topography


What did magellan search for in search of water across continent?
Route


In what world was all land west of 50 degrees w longitude assigned?
New


What did the papal decree of 1493 do to all land west of 50 degrees w longitude?
Assigned


What papal act of 1493 assigned all land in the new world west of 50 degrees w longitude to spain?
Decree


What did magellan search for in search of water?
Continent


Who set sail from spain with five ships in 1519?
Magellan


What was the purpose of magellan'